## Restaurant Evaluator

In [2]:
import json
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import io

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Part 1  

Shuyang specify data
Tokenize
Split data (80 training, 10 test, 10 validation)


#### Tokenization

#### Split Data

## Part 2

#### Build Neural Network

In [ ]:
model = tf.keras.Sequential([
    # Vectorize each word. The direction of which will be determined when the model is trained
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    # sum the vectors to get a value for the inputted sentence
    tf.keras.layers.GlobalAveragePooling1D(),
    # feed value into neural network
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

#### Train and Test the Model

In [ ]:
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

## Part 3  

Cornelius output a vector of probabilities (we only care about the probability that it's positive)
Validate.


#### Test the Model

#### Output Probability Vector

#### Validate the Model